In [1]:
import bs4 
import requests
import pandas as pd
import numpy as np
import os
import requests
import re
import datetime


In [2]:
now = datetime.datetime.now()
Today_date = now.strftime("%Y-%m-%d")

In [3]:
url = 'https://www.electronicsweekly.com/news/'
res = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
soup = bs4.BeautifulSoup(res.text,'lxml')

In [65]:
def listToString(s):   
    str1 = ""  
    
    for ele in s:  
        str1 += ele.text   
    return str1  
        

In [69]:
def get_text(url):
    res = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    soup = bs4.BeautifulSoup(res.text,'lxml')
    para = soup.find('div',{'class':'post-inner'}).find('div',{'class':'entry'}).findAll('p')
    text = listToString(para)
    
    return text

In [8]:
List = soup.find('li',{'id':'menu-item-445193'}).find('ul',{'class':'sub-menu'}).findAll('li')

In [11]:
List[0].find('a')

<a href="https://www.electronicsweekly.com/market-sectors/automotive-electronics/">Automotive Electronics</a>

In [43]:
records = []
for item in List:
    category = item.find('a').text
    url = item.find('a')['href']
    res = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    soup = bs4.BeautifulSoup(res.text,'lxml') 
    pages = soup.find('span',{'class':'pages'}).text.split()[-1]
    for i in range(0,int(pages)):
        
        res = requests.get(url+'/page/'+str(i),headers={'User-Agent': 'Mozilla/5.0'})
        soup = bs4.BeautifulSoup(res.text,'lxml') 
        sub_list = soup.findAll('article',{'class':'item-list'})

        for sub_item in sub_list:
            Source = 'Electronics Weekly'
            Category = category
            Heading = sub_item.find('h2',{'class':'post-title'}).find('a').text
            URL = sub_item.find('h2',{'class':'post-title'}).find('a')['href']
            Text = '-'
            records.append((Source,Heading,Category,URL,Text))

In [44]:
df = pd.DataFrame(records,columns=['Source','Heading','Category','URL','Text'])

In [46]:
#df.to_csv("Electronics Daily",index=False)

In [47]:
df

,Source,Heading,Category,URL,Text
0,Electronics Weekly,Chip inductors for automotive noise suppression,Automotive Electronics,https://www.electronicsweekly.com/news/product...,-
1,Electronics Weekly,Forbidden heat makes vehicle battery charge in...,Automotive Electronics,https://www.electronicsweekly.com/news/researc...,-
2,Electronics Weekly,Noise tolerant comparators work over wide temp...,Automotive Electronics,https://www.electronicsweekly.com/news/product...,-
3,Electronics Weekly,Suite of tools addresses ASIL-D safety design,Automotive Electronics,https://www.electronicsweekly.com/news/design/...,-
4,Electronics Weekly,GaN FET aids vehicle lidar systems’ accuracy,Automotive Electronics,https://www.electronicsweekly.com/market-secto...,-
...,...,...,...,...,...
3585,Electronics Weekly,Lambda lifts efficiency in off-line switching ...,Power,https://www.electronicsweekly.com/market-secto...,-
3586,Electronics Weekly,UK university adds to power engineering,Power,https://www.electronicsweekly.com/market-secto...,-
3587,Electronics Weekly,UK timing scheme cuts power and skew,Power,https://www.electronicsweekly.com/market-secto...,-
3588,Electronics Weekly,Power firm gets wafers in Europe,Power,https://www.electronicsweekly.com/market-secto...,-


In [73]:
df['Text'] = df['URL'].apply(get_text)

In [75]:
df.to_csv("Electronics Daily.csv",index=False)
